In [4]:
import kaggle
import pandas
import os
import re
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings

from torchvision.models import efficientnet_b0
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import numpy as np

from model.model import efficient_net_b0
from misc.train import train_model, test_model
from misc.plotting import get_examples
from misc.data_preprocessing import CustomImageDataset, get_statistics, download_data, create_dataloaders, get_dataloader_example


In [ ]:
path = './data/'
download_data(path)

In [ ]:
get_examples(5)

In [ ]:
# Train Statistics
get_statistics('./data/training_set/training_set/')

# Test Statistics
get_statistics('./data/test_set/test_set/')

In [ ]:
train_loader, val_loader, test_loader = create_dataloaders(
	train_directory_path="./data/training_set/training_set/", 
	test_directory_path="./data/test_set/test_set/", 
	batch_size=2,
	split=0.8
)

get_dataloader_example(train_loader)

In [ ]:
# Our efficient net model
model = efficient_net_b0()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f'Total number of parameters in our EfficientNet-B0: {total_params}')
 
# Built in pytorch efficient net model
model = efficientnet_b0(num_classes=2, pretrained=False)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f'Total number of parameters in official EfficientNet-B0: {total_params}')

In [10]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = efficient_net_b0()
model = model.to(device)

model_path = './model/model_weights.pth'
'''
if os.path.isfile(model_path):
	model.load_state_dict(torch.load(model_path)) 
	print('Model weights loaded from :', model_path)
else:
	print('No model weights were loaded')
'''

optimizer = torch.optim.Adam(
	params=model.parameters(),
	lr=1e-4
)

scheduler = ReduceLROnPlateau(
	optimizer,
	'min',
	factor=0.1,
	min_lr=1e-6
)

In [ ]:
train_model(
	model,
	optimizer,
	scheduler=scheduler,
	train_loader=train_loader,
	val_loader=val_loader,
	model_path=model_path,
	epochs=200,
	patience=25,
	threshold=1e-4
)

In [ ]:
model.load_state_dict(torch.load(model_path, map_location=device))

test_model(model, test_loader)

In [ ]:


import zipfile

# Path to your zip file
zip_file_path = 'dogs_vs_cats_2.zip'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('dogs_vs_cats_2')  # Specify the extraction path